In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
first_row = session.query(Measurement).first()
first_row.__dict__

In [ ]:
first_row = session.query(Station).first()
first_row.__dict__

In [ ]:
for row in session.query(Station.id, Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation).limit(15).all():
    print(row)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

In [ ]:
for row in session.query(Measurement.id, Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).order_by(Measurement.date.desc()).limit(15).all():
    print(row)

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
last_date = dt.datetime(2017, 8, 23)
one_year_date = last_date - dt.timedelta(days=365)
print("Last 12 Months: ", one_year_date)

In [ ]:
# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_date).all()
for result in results:
    print(f"Date: {result.date}, Precipitation: {result.prcp}")

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns=['date', 'precipitation'])
df.set_index('date', inplace=True, )
# Sort the dataframe by date
df=df.sort_index()
df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
df.plot(rot=90, legend=False)
plt.yticks(np.arange(0, 7, step=0.5))
plt.title('Precipitation')
plt.ylabel('Inches')
plt.xlabel('Date')
plt.figure(figsize=(6, 6))
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

In [ ]:
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_table=session.query(func.count(Station.station)).all()
station_table

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?  
# List the stations and the counts in descending order.
sel = [Measurement.station, 
       func.count(Measurement.station)
      ]
station_count = session.query(*sel).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()
station_count

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?   #USC00519281
sel = [Measurement.station, 
       func.min(Measurement.tobs), 
       func.max(Measurement.tobs), 
       func.avg(Measurement.tobs) 
       ]
station_temp = session.query(*sel).\
    filter(Measurement.station == "USC00519281").all()
station_temp


In [ ]:
#Design a query to retrieve the last 12 months of temperature observation data (TOBS).

date1 = dt.datetime(2016, 8, 23)

sel = [Measurement.station, 
       func.count(Measurement.station) 
      ]
year_temp = session.query(*sel).\
    filter(Measurement.date >= date1).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()
year_temp

#Filter by the station with the highest number of observations.

In [ ]:
#Plot the results as a histogram with `bins=12`.
sel = [Measurement.date, Measurement.tobs]
temp_year = session.query(*sel).\
    filter(Measurement.date >= one_year_date).filter(Measurement.station == "USC00519281").all()
#temp_year

In [ ]:
temp_year_df = pd.DataFrame(temp_year, columns=['date', 'tobs'])
temp_year_df.set_index('date', inplace=True, )
# Sort the dataframe by date
temp_year_df=temp_year_df.sort_index()
#temp_year_df

In [ ]:
ax = temp_year_df.plot.hist(bins=12, alpha=0.5)

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Get a list of column names and types
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
 session.query(Measurement.station, Station.station).limit(10).all()

In [ ]:
 same_station = session.query(Measurement, Station).filter(Measurement.station == Station.station).limit(10).all()

for record in same_station:
    (measurement, station) = record
    print(measurement.station)
    print(station.station)

In [ ]:
sel = [Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs, Station.latitude, Station.longitude, Station.elevation]
same_station = session.query(*sel).filter(Measurement.station == Station.station).limit(10).all()


        